# Train LSTM model with PCA features

Model LSTM: 4 layers, 256 units, 0.3 dropout

In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [8]:
import os

def save_model(model, history, base_dir_model, bar_dir_history, model_name, sub_model_name=""):

    # Tạo thư mục chính nếu chưa tồn tại
    if not os.path.exists(base_dir_model):
        os.makedirs(base_dir_model)
    if not os.path.exists(bar_dir_history):
        os.makedirs(bar_dir_history)

    model_file = os.path.join(base_dir_model, f"model_{model_name}_{sub_model_name}.h5")
    history_file = os.path.join(bar_dir_history, f"history_{model_name}_{sub_model_name}.pkl")

    # Lưu model
    model.save(model_file)
    print(f"Model saved to: {model_file}")

    # Lưu lịch sử huấn luyện
    with open(history_file, "wb") as f:
        pickle.dump(history.history, f)
    print(f"Model history saved to: {history_file}")



In [9]:
def build_lstm_model(input_shape, output_shape, number_UnitsLSTM):
    model = Sequential()
    for i in range (4):
        model.add(LSTM(number_UnitsLSTM, input_shape=input_shape, return_sequences=True))
        model.add(Dropout(0.3))
    model.add(Dense(number_UnitsLSTM, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    return model

def train_lstm_model(X_train, Y_train, X_val, Y_val, number_UnitsLSTM, batch_size=32, epochs=10):
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]), len(np.unique(Y_train)),number_UnitsLSTM)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, Y_val), callbacks=[early_stopping], verbose=1)
    return model, history

In [10]:
def prepareData(dir_train, dir_test):
    data_train = pd.read_csv(dir_train)
    data_test = pd.read_csv(dir_test)

    X_train = data_train.drop(['Activity', 'Activity_code'], axis=1)
    y_train = data_train['Activity_code']

    X_test = data_test.drop(['Activity', 'Activity_code'], axis=1)
    y_test = data_test['Activity_code']

    return X_train, X_test, y_train, y_test

dir_train = "../../data/interim/trainFinal.csv"
dir_test = "../../data/interim/testFinal.csv"
X_train, X_test, y_train, y_test = prepareData(dir_train, dir_test)

In [16]:
dir_train = "../../data/interim/train_PCA.csv"
dir_test = "../../data/interim/test_PCA.csv"

data_train_PCA = pd.read_csv(dir_train)
data_test_PCA = pd.read_csv(dir_test)
data_test_PCA = data_test_PCA.to_numpy()
data_train_PCA = data_train_PCA.to_numpy()
data_train_PCA = data_train_PCA.reshape((data_train_PCA.shape[0], 1, data_train_PCA.shape[1]))
data_test_PCA = data_test_PCA.reshape((data_test_PCA.shape[0], 1, data_test_PCA.shape[1]))

In [18]:
# Train LSTM model
epochs = 50
model_list_2 = []
model_history_list_2 = []
base_dir_model = "Experiment_Tracking_LSTM/models"
bar_dir_history = "Experiment_Tracking_LSTM/history_models"

model, history = train_lstm_model(data_train_PCA, y_train, data_test_PCA, y_test, epochs=epochs, number_UnitsLSTM=256)
model_name = "Exp4"
sub_model_name = "LSTM_with_PCA_dataset"
save_model(model, history, base_dir_model, bar_dir_history, model_name, sub_model_name)



Epoch 1/50
230/230 [==============================] - 14s 40ms/step - loss: 0.6227 - accuracy: 0.6961 - val_loss: 0.2560 - val_accuracy: 0.9192
Epoch 2/50
230/230 [==============================] - 2s 9ms/step - loss: 0.1061 - accuracy: 0.9626 - val_loss: 0.2146 - val_accuracy: 0.9318
Epoch 3/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0695 - accuracy: 0.9735 - val_loss: 0.2212 - val_accuracy: 0.9382
Epoch 4/50
230/230 [==============================] - 2s 10ms/step - loss: 0.0522 - accuracy: 0.9801 - val_loss: 0.2809 - val_accuracy: 0.9203
Epoch 5/50
230/230 [==============================] - 3s 11ms/step - loss: 0.0444 - accuracy: 0.9845 - val_loss: 0.3385 - val_accuracy: 0.9250
Epoch 6/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0342 - accuracy: 0.9867 - val_loss: 0.2851 - val_accuracy: 0.9352
Epoch 7/50
230/230 [==============================] - 2s 9ms/step - loss: 0.0225 - accuracy: 0.9916 - val_loss: 0.2876 - val_accuracy: 0.9457
Ep

In [21]:
max(history.history['val_accuracy'])

0.9457074999809265